In [1]:
import os
import sys
import random
from langchain_community.vectorstores import Chroma
from langchain_experimental.utilities import PythonREPL
from IPython.display import display, Markdown
import jsonlines
import chromadb
import yaml

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval
from utils.agents.static_RAG_with_coding import CodeRAG

examples = []

CONFIG_PATH = os.path.join(kit_dir, "config.yaml")

/Users/codym/CAP/aisk_forked/ai-starter-kit/comp_rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_config(filename: str) -> dict:
    try:
        with open(filename, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"The YAML configuration file {filename} was not found."
        )
    except yaml.YAMLError as e:
        raise RuntimeError(f"Error parsing YAML file: {e}")

In [3]:
CONFIG_PATH

'/Users/codym/CAP/aisk_forked/ai-starter-kit/agent_workflows/config.yaml'

In [4]:
configs = load_config(CONFIG_PATH)

In [5]:
configs["prompts"]["router_prompt"]

'prompt_engineering/prompts/llama3_8b-prompt_engineering-subquery_router.yaml'

In [6]:
documentRetrieval = DocumentRetrieval()
*_, embedding_model_info, retrieval_info, _ = documentRetrieval.get_config_info()
embeddings = documentRetrieval.load_embedding_model()

2024-07-10 15:11:05,899 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-07-10 15:11:06,756 [INFO] - Use pytorch device: cpu


max_seq_length  512


In [7]:
# chroma_client = chromadb.Client()
# collection = chroma_client.get_collection("/data/uber_lyft.chromadb")
vectorstore = Chroma(
    persist_directory=kit_dir + "/data/uber_lyft.chromadb",
    embedding_function=embeddings,
)

2024-07-10 15:11:06,770 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [8]:
# # count = vectorstore.count()
# count = len(documents["metadatas"])
# batch_size = 10

# for i in range(0, count, batch_size):
#     batch = vectorstore.get(include=["metadatas"], limit=batch_size, offset=i)

#     updated_metadatas = []
#     for metadata in batch["metadatas"]:
#         # print(metadata["filename"])
#         new_key = metadata["filename"].split("_")[0]
#         print(new_key)
#         metadata["filename"] = new_key
#         print(metadata["filename"])
#         updated_metadatas.append(metadata)

#     # vectorstore.update_documents(ids=batch["ids"], metadatas=updated_metadatas)

In [9]:
# instantiate rag
rag = CodeRAG(
    configs=CONFIG_PATH,
    embeddings=embeddings,
    vectorstore=vectorstore,
    examples=examples,
)

# Initialize chains
rag.initialize()

# Build nodes
workflow = rag.create_rag_nodes()
print(workflow)

# Build graph
app = rag.build_rag_graph(workflow)

AttributeError: 'CodeRAG' object has no attribute 'init_aggregation_chain'

In [ ]:
rag.display_graph(app)

In [ ]:
def log_answers(question, output_file):
    try:
        response = rag.call_rag(app, question)
    except:
        response = {}
        response["answer"] = "Failed"

    display(Markdown("---Response---"))
    display(Markdown(response["answer"]))

    if not os.path.exists(kit_dir + "/outputs"):
        os.makedirs(kit_dir + "/outputs")
    output_dir = kit_dir + "/outputs"

    with jsonlines.open(output_dir + f"/{output_file}", "a") as writer:
        writer.write({"question": question, "response": response["answer"]})

In [ ]:
questions = []